#Your Guide to Managing the Moral Sense Test

*Please note that, although we can obviously link to surveys through Qualtrics and other 3rd-party software, it is STRONGLY PREFERRED that you use code that can interface with SQL-- for instance, Javascript or Flash.*

## How to add a Qualtrics study....

There are **four** places in the html that you need to add you link to. All of these can be found in our webfaction server space. To find these pages, first SFTP in to the server (see Morgan for log in credentials). Then, navigate to webapps > phpmst. This is where the live webpages live.

1. index.html
2. user.php
3. studies.html
4. templates/studies_user.php

Studies can always be found in the **SLIDE 2** section of the page. They are formatted to be displayed in rows of five. Index.html and user.php should only display the top five "most popular" studies, while studies.html and studies_user.php are meant to be full lists of all the studies we have on the website at that time! So, if you're adding a study to index and user, you should simply replace one of the current studies. However, when adding to the studies pages, you can simply add yours as the last in line. You may need to make a new row so that the formatting stays pretty.

The code will look something like this:

In [ ]:
<div class="col-12 col-sm-2 with-hover-text">
	<p><a target="_blank" href="harvard.qualtrics.edu/MYSTUDYLINK"><i class="icon icon-map-marker"></i></p>
	<h2 class="font-semibold">STUDY NAME</h2>
	<h4 class="font-thin">Short enticing blurb here</h4>
	<h4 class="font-semibold">estimated time: 10 minutes</h4></a>
</div><!-- /col12 -->

Note that there are lots of icons to choose from-- try to pick one that provides some visual variation on the page! The full list can be found at the end of this tutorial.

That's all! Adding a study with Qualtrics is pretty painless, but it also doesn't have the advantages of full integration with our system. You won't get demographics or cross-correlations with other measures from users.

##How to add a Javascript or Flash study...

###Frontend


There are **four** places in the html that you need to add you link to. All of these can be found in our webfaction server space. To find these pages, first SFTP in to the server (see Morgan for log in credentials). Then, navigate to webapps > phpmst. This is where the live webpages live.

1. index.html
2. user.php
3. studies.html
4. templates/studies_user.php

Studies can always be found in the **SLIDE 2** section of the page. They are formatted to be displayed in rows of five. Index.html and user.php should only display the top five "most popular" studies, while studies.html and studies_user.php are meant to be full lists of all the studies we have on the website at that time! So, if you're adding a study to index and user, you should simply replace one of the current studies. However, when adding to the studies pages, you can simply add yours as the last in line. You may need to make a new row so that the formatting stays pretty.

The code will look something like this:

In [ ]:
<div class="col-12 col-sm-2 with-hover-text">
	<p><a target="_blank" href="turkgate.php?studyname"><i class="icon icon-map-marker"></i></p>
	<h2 class="font-semibold">STUDY NAME</h2>
	<h4 class="font-thin">Short enticing blurb here</h4>
	<h4 class="font-semibold">estimated time: 10 minutes</h4></a>
</div><!-- /col12 -->

Note that there are lots of icons to choose from-- try to pick one that provides some visual variation on the page! The full list can be found at the end of this tutorial.

We use turkgate.php to check whether or not users (or visitors, within the same session) have already taken this or a similar test. If not, it does one of two things:
1. IF ITS NOT A REGISTERED USER: Directs them to **visitordem.php?studyname**, which collects visitor's demographic information and then redirects them to studyname.html. So, if my experiment lives on empathy.html, my link should be turkgate.php?empathy
2. IF IT IS A USER: Directs them to the study at studyname.html without going through demographics.

**If you need TurkGate functionality amongst a group of studies (rather than just making sure no one is taking the same survey twice), please see Morgan.**

###Setting up the SQL table

Although you're welcome to use your own SQL set up if you're more comfortable doing it that way, again I urge you to follow this tutorial so that the lab can get the most out of this data.

First of all, see Morgan for the log in credentials to the SQL database. There's a fairly good interface that we can use, which can be accessed at [this link](web402.webfaction.com/phpMyAdmin/index.php), if you do not have a preferred SQL client.

For now, while we still have a manageable number of studies and tables, I'd prefer that all MST surveys have their data tables **in the database "mst".** Why? Well, it helps us efficiently insert information in to multiple different tables at once (the way we must in order to keep demographics separate from responses separate from identifying information). **The name of your data table should match the name you used for your experiment web page.** If my survey is at empathy.html, the table name should be empathy.

**Your table should have the following columns:**
1. rt
2. response
3. scenNum (if you're using randomization)
4. trialNum
5. userid
6. visitorid
7. timestamp (time of row insertion)

*Feel free to copy the structure of an existing table in the database. You can do this by clicking on one of the tables (such as empathy) and clicking on More > Operations in the top right corner. The fourth cell of that page will give you the option to copy table structure into a new table with your desired name. Even if you decide not to do this, **please check that your data types are consistent**.*

You'll need to do one more thing in SQL. Go to the table **useremails** and add a column with the name of your study. This column should hold a boolean value that defaults to FALSE or 0. This is the table we use to record which users have taken which surveys. Every time a given user takes a new survey, we look up their email address, and revise the 0 in the test's column to a 1.

###Saving experiment data to SQL

I have added several templates to this repo to illustrate a working experiment. There are three relevant files: **studyname.html**, **parameters_studyname.js** and **saveDataSTUDYNAME.php**. Here's how they work together.

The *parameters* file contains information such as the text of the scenarios you'll be showing (if it's a standard vignette study) and number of trials. The *parameters* file is read in at the top incantation of *studyname.html*. This file is the actual UI for the experiment. My template uses jsPsych to render a simple likert experiment. Consent and debrief, as well as a function that provides feedback to the user (**strongly recommended!**) are included in this template. studyname.html collects data from the experiment, and upon finish sends the data, the name of the SQL table, and variables (such as randomized order) to *saveDataSTUDYNAME.php*. This final script parses the data and collects the session variables (userid and visitorid) in order to generate rows to be inserted into the SQL table specified. Finally, it connects to our database and inserts one row per trial. If the test-taker is a user with an email on file, it also flips the boolean for the current study in the **useremails** database.

**Okay, now that I've got my scripts working, where do I put them?**

1. The studyname.html file goes in webapps/phpmst/experiment. 
2. The parameters_studyname.js file goes in webapps/phpmst/experiment/public/js
3. The saveDataSTUDYNAME.php file goes in webapps/phpmst/experiment/data

More information is of course available in the commented code of these templates. Although everyone's javascript interface will be a little bit different, I do think the saveData function will stay essentially the same. 

The most challenging thing about this workflow is accessing the data. jsPsych has a pretty clunky data structure, so you might need to do some debugging and finagling to get your responses to record correctly. See [here](http://docs.jspsych.org/core_library/jspsych-data/) and [here](https://groups.google.com/forum/#!forum/jspsych) for documentation relating to this. 

##Useful SQL Commands and Tidbits

#

##Full list of available icons

.icon-glass
.icon-music
.icon-search
.icon-envelope-alt
.icon-heart
.icon-star
.icon-star-empty
.icon-user
.icon-film
.icon-th-large
.icon-th
.icon-th-list
.icon-ok
.icon-remove
.icon-zoom-in
.icon-zoom-out
.icon-power-off
.icon-signal
.icon-gear
.icon-trash
.icon-home
.icon-file-alt
.icon-time
.icon-road
.icon-download-alt
.icon-download
.icon-upload
.icon-inbox
.icon-play-circle
.icon-rotate-right
.icon-refresh
.icon-list-alt
.icon-lock
.icon-flag
.icon-headphones
.icon-volume-off
.icon-volume-down
.icon-volume-up
.icon-qrcode
.icon-barcode
.icon-tag
.icon-tags
.icon-book
.icon-bookmark
.icon-print
.icon-camera
.icon-font
.icon-bold
.icon-italic
.icon-text-height
.icon-text-width
.icon-align-left
.icon-align-center
.icon-align-right
.icon-align-justify
.icon-list
.icon-indent-left
.icon-indent-right
.icon-facetime-video
.icon-picture
.icon-pencil
.icon-map-marker
.icon-adjust
.icon-tint
.icon-edit
.icon-share
.icon-check
.icon-move
.icon-step-backward
.icon-fast-backward
.icon-backward
.icon-play
.icon-pause
.icon-stop
.icon-forward
.icon-fast-forward
.icon-step-forward
.icon-eject
.icon-chevron-left
.icon-chevron-right
.icon-plus-sign
.icon-minus-sign
.icon-remove-sign
.icon-ok-sign
.icon-question-sign
.icon-info-sign
.icon-screenshot
.icon-remove-circle
.icon-ok-circle
.icon-ban-circle
.icon-arrow-left
.icon-arrow-right
.icon-arrow-up
.icon-arrow-down
.icon-mail-forward
.icon-resize-full
.icon-resize-small
.icon-plus
.icon-minus
.icon-asterisk
.icon-exclamation-sign
.icon-gift
.icon-leaf
.icon-fire
.icon-eye-open
.icon-eye-close
.icon-warning-sign
.icon-plane
.icon-calendar
.icon-random
.icon-comment
.icon-magnet
.icon-chevron-up
.icon-chevron-down
.icon-retweet
.icon-shopping-cart
.icon-folder-close
.icon-folder-open
.icon-resize-vertical
.icon-resize-horizontal
.icon-bar-chart
.icon-twitter-sign
.icon-facebook-sign
.icon-camera-retro
.icon-key
.icon-gears
.icon-comments
.icon-thumbs-up-alt
.icon-thumbs-down-alt
.icon-star-half
.icon-heart-empty
.icon-signout
.icon-linkedin-sign
.icon-pushpin
.icon-external-link
.icon-signin
.icon-trophy
.icon-github-sign
.icon-upload-alt
.icon-lemon
.icon-phone
.icon-unchecked
.icon-bookmark-empty
.icon-phone-sign
.icon-twitter
.icon-facebook
.icon-github
.icon-unlock
.icon-credit-card
.icon-rss
.icon-hdd
.icon-bullhorn
.icon-bell
.icon-certificate
.icon-hand-right
.icon-hand-left
.icon-hand-up
.icon-hand-down
.icon-circle-arrow-left
.icon-circle-arrow-right
.icon-circle-arrow-up
.icon-circle-arrow-down
.icon-globe
.icon-wrench
.icon-tasks
.icon-filter
.icon-briefcase
.icon-fullscreen
.icon-group
.icon-link
.icon-cloud
.icon-beaker
.icon-cut
.icon-copy
.icon-paperclip
.icon-save
.icon-sign-blank
.icon-reorder
.icon-list-ul
.icon-list-ol
.icon-strikethrough
.icon-underline
.icon-table
.icon-magic
.icon-truck
.icon-pinterest
.icon-pinterest-sign
.icon-google-plus-sign
.icon-google-plus
.icon-money
.icon-caret-down
.icon-caret-up
.icon-caret-left
.icon-caret-right
.icon-columns
.icon-sort
.icon-sort-down
.icon-sort-up
.icon-envelope
.icon-linkedin
.icon-rotate-left
.icon-legal
.icon-dashboard
.icon-comment-alt
.icon-comments-alt
.icon-bolt
.icon-sitemap
.icon-umbrella
.icon-paste
.icon-lightbulb
.icon-exchange
.icon-cloud-download
.icon-cloud-upload
.icon-user-md
.icon-stethoscope
.icon-suitcase
.icon-bell-alt
.icon-coffee
.icon-food
.icon-file-text-alt
.icon-building
.icon-hospital
.icon-ambulance
.icon-medkit
.icon-fighter-jet
.icon-beer
.icon-h-sign
.icon-plus-sign-alt
.icon-double-angle-left
.icon-double-angle-right
.icon-double-angle-up
.icon-double-angle-down
.icon-angle-left
.icon-angle-right
.icon-angle-up
.icon-angle-down
.icon-desktop
.icon-laptop
.icon-tablet
.icon-mobile-phone
.icon-circle-blank
.icon-quote-left
.icon-quote-right
.icon-spinner
.icon-circle
.icon-mail-reply
.icon-github-alt
.icon-folder-close-alt
.icon-folder-open-alt
.icon-expand-alt
.icon-collapse-alt
.icon-smile
.icon-frown
.icon-meh
.icon-gamepad
.icon-keyboard
.icon-flag-alt
.icon-flag-checkered
.icon-terminal
.icon-code
.icon-reply-all
.icon-mail-reply-all
.icon-star-half-full
.icon-location-arrow
.icon-crop
.icon-code-fork
.icon-unlink
.icon-question
.icon-info
.icon-exclamation
.icon-superscript
.icon-subscript
.icon-eraser
.icon-puzzle-piece
.icon-microphone
.icon-microphone-off
.icon-shield
.icon-calendar-empty
.icon-fire-extinguisher
.icon-rocket
.icon-maxcdn
.icon-chevron-sign-left
.icon-chevron-sign-right
.icon-chevron-sign-up
.icon-chevron-sign-down
.icon-html5
.icon-css3
.icon-anchor
.icon-unlock-alt
.icon-bullseye
.icon-ellipsis-horizontal
.icon-ellipsis-vertical
.icon-rss-sign
.icon-play-sign
.icon-ticket
.icon-minus-sign-alt
.icon-check-minus
.icon-level-up
.icon-level-down
.icon-check-sign
.icon-edit-sign
.icon-external-link-sign
.icon-share-sign
.icon-compass
.icon-collapse
.icon-collapse-top
.icon-expand
.icon-euro
.icon-gbp
.icon-dollar
.icon-rupee
.icon-yen
.icon-renminbi
.icon-won
.icon-bitcoin
.icon-file
.icon-file-text
.icon-sort-by-alphabet
.icon-sort-by-alphabet-alt
.icon-sort-by-attributes
.icon-sort-by-attributes-alt
.icon-sort-by-order
.icon-sort-by-order-alt
.icon-thumbs-up
.icon-thumbs-down
.icon-youtube-sign
.icon-youtube
.icon-xing
.icon-xing-sign
.icon-youtube-play
.icon-dropbox
.icon-stackexchange
.icon-instagram
.icon-flickr
.icon-adn
.icon-bitbucket
.icon-bitbucket-sign
.icon-tumblr
.icon-tumblr-sign
.icon-long-arrow-down
.icon-long-arrow-up
.icon-long-arrow-left
.icon-long-arrow-right
.icon-apple
.icon-windows
.icon-android
.icon-linux
.icon-dribbble
.icon-skype
.icon-foursquare
.icon-trello
.icon-female
.icon-male
.icon-gittip
.icon-sun
.icon-moon
.icon-archive
.icon-bug
.icon-vk
.icon-weibo
.icon-renren